# Bakeoff

### Authors:
* Arthur Kim
* Douglas Lu
* Shane Mangold
* Nate Walter

## Import

Here we imported all the relevant libraries for the project

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence as influence

## Review the Data File

The data file was reviewed to get a better understanding of what features were included and what data types are the features

In [2]:
#read in the file
X_train = pd.read_csv('bakeoff_data/Xtrain.csv')
y_train = pd.read_csv('bakeoff_data/ytrain.csv')
X_test = pd.read_csv('bakeoff_data/Xtest.csv')

In [3]:
#check the first 5 entries in the data
X_train.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3/4/2015,3,2.50,1880,4499,2.0,0.0,0.0,3,8,1880,0.0,1993,0.0,98029,47.5664,-121.999,2130,5114
1,10/7/2014,3,2.50,2020,6564,1.0,0.0,0.0,3,7,1310,710.0,1994,0.0,98042,47.3545,-122.158,1710,5151
2,1/16/2015,5,4.00,4720,493534,2.0,0.0,0.0,5,9,3960,760.0,1975,0.0,98027,47.4536,-122.009,2160,219542
3,3/30/2015,2,2.00,1430,3880,1.0,0.0,0.0,4,7,1430,0.0,1949,0.0,98117,47.6844,-122.392,1430,3880
4,10/14/2014,3,2.25,2270,32112,1.0,0.0,0.0,4,8,1740,530.0,1980,0.0,98042,47.3451,-122.094,2310,41606


In [4]:
#check the columns and nulls
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16197 entries, 0 to 16196
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           16197 non-null  object 
 1   bedrooms       16197 non-null  int64  
 2   bathrooms      16197 non-null  float64
 3   sqft_living    16197 non-null  int64  
 4   sqft_lot       16197 non-null  int64  
 5   floors         16197 non-null  float64
 6   waterfront     14441 non-null  float64
 7   view           16148 non-null  float64
 8   condition      16197 non-null  int64  
 9   grade          16197 non-null  int64  
 10  sqft_above     16197 non-null  int64  
 11  sqft_basement  16197 non-null  object 
 12  yr_built       16197 non-null  int64  
 13  yr_renovated   13318 non-null  float64
 14  zipcode        16197 non-null  int64  
 15  lat            16197 non-null  float64
 16  long           16197 non-null  float64
 17  sqft_living15  16197 non-null  int64  
 18  sqft_l

In [5]:
#check the first 5 entries in the data
y_train.head()

,price
0,529000.0
1,253000.0
2,745000.0
3,545000.0
4,390000.0


In [6]:
#check the first 5 entries in the data
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16197 entries, 0 to 16196
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   16197 non-null  float64
dtypes: float64(1)
memory usage: 126.7 KB


## Cleaning the Data

After reviewing the data, we noticed that some of the columns had nulls or the wrong data type. For the nulls, we made assumptions to fill in the null values. The column "sqft_basement" had to be converted to a number.

In [7]:
train_df = pd.concat([X_train, y_train], axis=1)

In [8]:
#for year renovated, convert any houses that have been renovated to '1' to indicate true
#for any nulls, assume no renovation
train_df['yr_renovated'].fillna(0, inplace=True)
train_df['yr_renovated'] = train_df['yr_renovated'].apply(lambda x: 1 if x > 0 else x)
X_test['yr_renovated'].fillna(0, inplace=True)
X_test['yr_renovated'] = X_test['yr_renovated'].apply(lambda x: 1 if x > 0 else x)

In [9]:
train_df.rename(columns={'yr_renovated': 'if_renovated'}, inplace=True)
X_test.rename(columns={'yr_renovated': 'if_renovated'}, inplace=True)

In [10]:
#for any nulls, assume no waterfront
train_df['waterfront'].fillna(0, inplace=True)
X_test['waterfront'].fillna(0, inplace=True)

In [11]:
#for any nulls, assume no one viewed the property
train_df['view'].fillna(0, inplace=True)
X_test['view'].fillna(0, inplace=True)

In [12]:
#clean up sqft_basement and convert to int
train_df['sqft_basement'] = train_df['sqft_basement'].replace({'?':np.nan}).astype(float)
train_df['sqft_basement'].fillna(train_df['sqft_living']-train_df['sqft_above'], inplace=True)
X_test['sqft_basement'] = X_test['sqft_basement'].replace({'?':np.nan}).astype(float)
X_test['sqft_basement'].fillna(X_test['sqft_living']-X_test['sqft_above'], inplace=True)

We also extracted the month from the date to see if there were any seasonal impact on housing prices.

In [13]:
#retrieve the months and year
train_df['month_of_date'] = pd.DatetimeIndex(train_df['date']).month
train_df['year_of_date'] = pd.DatetimeIndex(train_df['date']).year
X_test['month_of_date'] = pd.DatetimeIndex(X_test['date']).month
X_test['year_of_date'] = pd.DatetimeIndex(X_test['date']).year

We converted the year built column to the age of the house to compare the data more easily

In [14]:
#convert yr_built to age of house by subtracting year the property was sold by the year it was built
#to create a more sensible column 
train_df['age_of_house'] = train_df['year_of_date'] - train_df['yr_built']
X_test['age_of_house'] = X_test['year_of_date'] - X_test['yr_built']

#drop year of date because years are only 2014 and 2015, and will not impact our predicative model
#drop yr_built b/c it is redundant with age_of_house
train_df.drop(columns=['year_of_date'], inplace=True)
train_df.drop(columns=['yr_built'], inplace=True)
X_test.drop(columns=['year_of_date'], inplace=True)
X_test.drop(columns=['yr_built'], inplace=True)

In [15]:
#drop duplicates if any
train_df.drop_duplicates(inplace=True)
X_test.drop_duplicates(inplace=True)

In [16]:
#drop id and date columns
train_df.drop(columns=['date'], inplace=True)
X_test.drop(columns=['date'], inplace=True)

In [17]:
#reset index
train_df.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)

In [18]:
#convert some of the categorical data from floats to ints
train_df['waterfront'] = train_df['waterfront'].astype(int)
train_df['view'] = train_df['view'].astype(int)
train_df['sqft_basement'] = train_df['sqft_basement'].astype(int)
train_df['if_renovated'] = train_df['if_renovated'].astype(int)
X_test['waterfront'] = X_test['waterfront'].astype(int)
X_test['view'] = X_test['view'].astype(int)
X_test['sqft_basement'] = X_test['sqft_basement'].astype(int)
X_test['if_renovated'] = X_test['if_renovated'].astype(int)

In [19]:
X_train = train_df.drop('price', axis=1)
y_train = pd.DataFrame(train_df['price'])

In [20]:
#check cleaned data
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16197 entries, 0 to 16196
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       16197 non-null  int64  
 1   bathrooms      16197 non-null  float64
 2   sqft_living    16197 non-null  int64  
 3   sqft_lot       16197 non-null  int64  
 4   floors         16197 non-null  float64
 5   waterfront     16197 non-null  int64  
 6   view           16197 non-null  int64  
 7   condition      16197 non-null  int64  
 8   grade          16197 non-null  int64  
 9   sqft_above     16197 non-null  int64  
 10  sqft_basement  16197 non-null  int64  
 11  if_renovated   16197 non-null  int64  
 12  zipcode        16197 non-null  int64  
 13  lat            16197 non-null  float64
 14  long           16197 non-null  float64
 15  sqft_living15  16197 non-null  int64  
 16  sqft_lot15     16197 non-null  int64  
 17  month_of_date  16197 non-null  int64  
 18  age_of

In [21]:
X_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,if_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month_of_date,age_of_house
0,3,2.50,1880,4499,2.0,0,0,3,8,1880,0,0,98029,47.5664,-121.999,2130,5114,3,22
1,3,2.50,2020,6564,1.0,0,0,3,7,1310,710,0,98042,47.3545,-122.158,1710,5151,10,20
2,5,4.00,4720,493534,2.0,0,0,5,9,3960,760,0,98027,47.4536,-122.009,2160,219542,1,40
3,2,2.00,1430,3880,1.0,0,0,4,7,1430,0,0,98117,47.6844,-122.392,1430,3880,3,66
4,3,2.25,2270,32112,1.0,0,0,4,8,1740,530,0,98042,47.3451,-122.094,2310,41606,10,34


In [22]:
#check cleaned data
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5400 entries, 0 to 5399
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       5400 non-null   int64  
 1   bathrooms      5400 non-null   float64
 2   sqft_living    5400 non-null   int64  
 3   sqft_lot       5400 non-null   int64  
 4   floors         5400 non-null   float64
 5   waterfront     5400 non-null   int64  
 6   view           5400 non-null   int64  
 7   condition      5400 non-null   int64  
 8   grade          5400 non-null   int64  
 9   sqft_above     5400 non-null   int64  
 10  sqft_basement  5400 non-null   int64  
 11  if_renovated   5400 non-null   int64  
 12  zipcode        5400 non-null   int64  
 13  lat            5400 non-null   float64
 14  long           5400 non-null   float64
 15  sqft_living15  5400 non-null   int64  
 16  sqft_lot15     5400 non-null   int64  
 17  month_of_date  5400 non-null   int64  
 18  age_of_h

In [23]:
X_test.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,if_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month_of_date,age_of_house
0,3,0.75,850,8573,1.0,0,0,3,6,600,250,0,98146,47.5030,-122.356,850,8382,2,70
1,3,1.00,1510,6083,1.0,0,0,4,6,860,650,0,98115,47.6966,-122.324,1510,5712,10,74
2,4,2.25,1790,42000,1.0,0,0,3,7,1170,620,0,98045,47.4819,-121.744,2060,50094,3,32
3,2,1.50,1140,2500,1.0,0,1,3,7,630,510,0,98106,47.5707,-122.359,1500,5000,2,27
4,3,1.00,1500,3920,1.0,0,0,3,7,1000,500,0,98107,47.6718,-122.359,1640,4017,5,67


## Log-Transforming Data

To normalize the price data, we log-transformed (base of e) the data. We can see in the histogram below, the log-transformed data looks more similar to a normal distribution than untransformed price data. 

In [24]:
#Because the data is skewed to the right, transform the price data using log
y_train['price'] = np.log(y_train['price'])

In [25]:
y_train.head()

,price
0,13.178744
1,12.441145
2,13.521139
3,13.208541
4,12.873902


In [26]:
#based on the pairplot, we can see which data are categorical and which are numeric
numeric = ['bedrooms', 
           'bathrooms', 
           'sqft_living', 
           'sqft_lot', 
           'sqft_above', 
           'sqft_basement',
           'lat', 
           'long',
           'sqft_living15', 
           'sqft_lot15']

categorical = ['floors',
               'waterfront', 
               'view', 
               'condition', 
               'grade',
               'if_renovated',
               'zipcode',
               'month_of_date']

It's important to validate that all observations are properly input or they could throw off the modeling process. Some erroneous observations may have values that dont make sense in a realworld context making them easy to identify and remove. 

In [27]:
#visually inspecting value counts to look for weird values
for column in X_train.columns:
    display(X_train[column].value_counts())

3     7339
4     5184
2     2074
5     1199
6      199
1      152
7       33
8       10
9        4
10       2
11       1
Name: bedrooms, dtype: int64

2.50    4038
1.00    2878
1.75    2296
2.25    1528
2.00    1433
1.50    1097
2.75     879
3.00     571
3.50     550
3.25     441
4.00     111
3.75     109
4.50      80
4.25      64
0.75      53
4.75      19
5.00      14
5.25      10
1.25       5
5.50       5
0.50       4
6.00       3
5.75       3
6.50       2
8.00       2
6.75       2
Name: bathrooms, dtype: int64

1800    104
1300    103
1010    101
1540    100
1480     99
       ... 
4870      1
2885      1
5030      1
5070      1
1867      1
Name: sqft_living, Length: 896, dtype: int64

5000     264
6000     209
4000     189
7200     167
7500      96
        ... 
7847       1
5806       1
9904       1
18815      1
43017      1
Name: sqft_lot, Length: 8015, dtype: int64

1.0    8002
2.0    6185
1.5    1425
3.0     463
2.5     115
3.5       7
Name: floors, dtype: int64

...

-122.290    91
-122.362    83
-122.291    81
-122.288    77
-122.372    76
            ..
-122.519     1
-121.730     1
-122.460     1
-122.484     1
-122.448     1
Name: long, Length: 720, dtype: int64

1540    151
1560    145
1440    144
1480    129
1720    128
       ... 
2092      1
2075      1
2156      1
2236      1
4070      1
Name: sqft_living15, Length: 691, dtype: int64

5000     324
4000     260
6000     214
7200     156
7500     105
        ... 
7775       1
17771      1
5734       1
7783       1
8196       1
Name: sqft_lot15, Length: 7230, dtype: int64

5     1816
4     1682
7     1659
6     1604
8     1473
3     1422
10    1393
9     1340
12    1101
11    1060
2      909
1      738
Name: month_of_date, dtype: int64

 9      358
 10     331
 8      330
 11     314
 0      312
       ... 
 113     21
 115     19
 81      19
 80      17
-1       10
Name: age_of_house, Length: 117, dtype: int64

In [28]:
#log transform all numeric feature except:
#sqft basement - has values of 0
#long - has negative values
to_ln = ['bedrooms',
         'bathrooms',
         'sqft_living',
         'sqft_lot', 
         'sqft_above',
         'lat',
         'sqft_living15', 
         'sqft_lot15']

for column in to_ln:
    X_train[column] = np.log(X_train[column])
    X_test[column] = np.log(X_test[column])

## Data Mainipulation

For this section, we focused on create an efficient way to take our cleaned dataframe and create train and test splits for each of our model.

In [29]:
#instantiating OHE object
ohe = OneHotEncoder(sparse=False, handle_unknown='error', drop='first')

#Fitting object onto test and trasnforming test and train
X_train_ohe = ohe.fit_transform(X_train[categorical])
X_test_ohe = ohe.transform(X_test[categorical])

#placing column names onto our new categorical columns and formatting as df
X_train_ohe_df = pd.DataFrame(X_train_ohe, columns=ohe.get_feature_names(categorical), 
                              index=X_train.index)
X_test_ohe_df = pd.DataFrame(X_test_ohe, columns=ohe.get_feature_names(categorical),
                            index=X_test.index)

#combining categoricals with rest of data
X_train = pd.concat([X_train[numeric], X_train_ohe_df],axis=1)
X_test = pd.concat([X_test[numeric], X_test_ohe_df], axis=1)

#scaling train,test splits
X_list = [X_train,X_test]

ss=StandardScaler()
for i in X_list:
    ss.fit(i)
    i = pd.DataFrame(ss.transform(i))

## Formulas and Useful Objects

We created several formulas to efficiently run repetitive processes, including running cross validation, checking for R-squared and RMSE, and plotting different linear regression assumption tests.

In [30]:
def cross_val(estimator,X,y,n_splits=10,test_size=0.25, random_state=None):
    """
    This formula performs cross validation using shuffled splits. Output is a tuple,
    The 0th element is the median R2 score for the train sets, the 1st element
    is the median R2 score for the test sets.
    
    """
    splitter = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

    output = cross_validate(estimator, X=X, y=y, cv=splitter, return_train_score=True)
    return np.median(output['train_score']), np.median(output['test_score'])

In [31]:
#returns a summary of the median train R-squared, median test R-squared, and differential score based
#on the cross validation
def cval_summary(train,test,diff):
    return f"The median R-squared values for the train sets were {round(train,3)}, the median R-squared values for the test sets were {round(test,3)}. These values resulted in a differential of {round(diff,5)}"

In [32]:
def score_model(model, Xtrain, Xtest, ytrain, ytest, log=False):
    '''
    This function takes in a model and the train and test samples and returns
    the train R-squared, test R-squared, and the RMSE
    '''
    if log == False:
        rmse = mean_squared_error(ytest, model.predict(Xtest), squared=False)
    else:
        rmse = mean_squared_error(np.exp(ytest), np.exp(model.predict(Xtest)), squared=False)
    return model.score(Xtrain, ytrain),  model.score(Xtest, ytest), rmse

In [33]:
#returns a summary of the train R-squared, test R-squared, differential between R-squared, and RMSE
def model_summary(train,test,diff,rmse):
    return f"The R-squared value for the train set was {round(train,3)}, and the R-squared value for the test set was {round(test,3)}. These values resulted in a differential of {round(diff,5)}. The RMSE of our model predicitons was {round(rmse,2)}"

In [34]:
def normality_test(ols_model):
    """
    tests for normality by taking in an OLS model, and reporting out different test features
    and plots the Q-Q plot
    """
    residuals = ols_model.resid
    name = ['Jarque-Bera','Prob','Skew', 'Kurtosis']
    test = sms.jarque_bera(residuals)
    for name, test in zip(name, test):
        print('\n',name, '----')
        print(test)
    fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);
    fig.show();

In [35]:
def homoscedasticity_test(ols_model):
    """
    tests for homoscedasticity by taking in an OLS model, and reporting out different test features
    and plots residual vs fitted plot
    """
    predicted_y = ols_model.predict()
    resids = ols_model.resid

    fig, ax = plt.subplots()

    sns.regplot(x=predicted_y, y=resids, lowess=True, ax=ax, line_kws={'color': 'red'})
    ax.set_title('Residuals vs Fitted', fontsize=16)
    ax.set(xlabel='Fitted Values', ylabel='Residuals')

    bp_test = pd.DataFrame(sms.het_breuschpagan(resids, ols_model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, ols_model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plot ----')

In [36]:
def actual_vs_predicted(model,X_test,y_test):
    """
    Plots the actual y vs the predicted y
    """
    y_predicted = model.predict(X_test)
    fig, ax = plt.subplots(figsize=(10,10))
    ax.scatter(x=y_test, y=y_predicted)
    ax.set_xlabel("Actual Price Values")
    ax.set_ylabel("Predicted Price Values")
    ax.set_title("Actual vs Predicted")
    
    p1 = max(max(y_test), max(y_predicted))
    p2 = min(min(y_test), min(y_predicted))
    plt.plot([p1, p2], [p1, p2], 'b-')

In [37]:
def plot_cooks_distance(c):
    """
    takes in cook's distance data and plots the cook's distance
    """
    _, ax = plt.subplots(figsize=(6,6))
    ax.stem(c, markerfmt=",")
    ax.set_xlabel("instance")
    ax.set_ylabel("distance")
    ax.set_title("Cook's Distance")
    return ax

## Model 1

Model 1 uses a multiple linear regression model of the data using all features, including the dummied out categorical features.

### Model 1: Target Price

We analyzed the untransformed price data set.

In [38]:
#create linear regression model for price and setting up cross validation 
model1 = LinearRegression()
model1.fit(X_train, y_train);

In [39]:
#setting up cross validation for price in a different way 
model1_cval = cross_val(model1,
                        X= X_train,
                        y=y_train,
                        random_state=0)

model1_cval_summary = cval_summary(model1_cval[0],
                                   model1_cval[1],
                                   abs(model1_cval[0]-model1_cval[1]))
model1_cval_summary

'The median R-squared values for the train sets were 0.888, the median R-squared values for the test sets were 0.887. These values resulted in a differential of 0.00071'

In [40]:
y_hat = model1.predict(X_test)
y_hat = np.exp(y_hat)
y_hat.shape

(5400, 1)

In [41]:
np.savetxt('DS_060721_Group_1_bakeoff_results.csv', y_hat, delimiter=',')